# Live Camera

In [2]:
import tensorflow_hub as hub
import cv2
import numpy
import tensorflow as tf
import pandas as pd

# Carregar modelos
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
labels = pd.read_csv('labels.csv',sep=';',index_col='ID')
labels = labels['OBJECT (2017 REL.)']

cap = cv2.VideoCapture(0)
#cap = cv2.cvtColor(cap, cv2.COLOR_BGR2RGB)

width = 1280
height = 720
trackrange = [(),(),(),()]

while(True):
    #Capture frame-by-frame
    ret, frame = cap.read()
    
    #Resize to respect the input_shape
    inp = cv2.resize(frame, (width , height ))

    #Convert img to RGB


    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

    #Is optional but i recommend (float convertion and convert img to tensor image)
    rgb_tensor = tf.convert_to_tensor(rgb, dtype=tf.uint8)

    #Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor , 0)

    boxes, scores, classes, num_detections = detector(rgb_tensor)
 
    pred_labels = classes.numpy().astype('int')[0]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]

    # Extract only boxes and labels corresponding to 'person' class
    person_boxes = []
    person_labels = []
    for score, (ymin,xmin,ymax,xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
        if labels[label] == 'person':
            person_boxes.append((ymin, xmin, ymax, xmax))
            person_labels.append(labels[label])

    # loop throughout the person detections and place a box around it
    for box, label in zip(person_boxes, person_labels):
        ymin, xmin, ymax, xmax = box
        img_boxes = cv2.rectangle(inp, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)      
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_boxes, label, (xmin, ymax-10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    
    #Display the resulting frame
    cv2.imshow('black and white', img_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


In [1]:
import tensorflow_hub as hub
import cv2
import numpy as np
import tensorflow as tf
import pandas as pd

# Carregar modelos
detector = hub.load("https://tfhub.dev/tensorflow/efficientdet/lite2/detection/1")
labels = pd.read_csv('labels.csv', sep=';', index_col='ID')
labels = labels['OBJECT (2017 REL.)']

cap = cv2.VideoCapture(0)

width = 720
height = 720

# Define the ROI mask
mask = np.zeros((height, width), dtype=np.uint8)
roi_poly = np.array([[(240, 240), (480, 240), (480, 480), (240, 480)]])
cv2.fillPoly(mask, roi_poly, 255)

img_boxes = None

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Resize to respect the input_shape
    inp = cv2.resize(frame, (width, height))

    # Convert img to RGB
    rgb = cv2.cvtColor(inp, cv2.COLOR_BGR2RGB)

    # Apply the ROI mask to the input image
    masked_rgb = cv2.bitwise_and(rgb, rgb, mask=mask)

    # Is optional but i recommend (float convertion and convert img to tensor image)
    rgb_tensor = tf.convert_to_tensor(masked_rgb, dtype=tf.uint8)

    # Add dims to rgb_tensor
    rgb_tensor = tf.expand_dims(rgb_tensor, 0)

    boxes, scores, classes, num_detections = detector(rgb_tensor)

    pred_labels = classes.numpy().astype('int')[0]
    pred_boxes = boxes.numpy()[0].astype('int')
    pred_scores = scores.numpy()[0]

    # Extract only boxes and labels corresponding to 'person' class within ROI
    person_boxes = []
    person_labels = []
    for score, (ymin, xmin, ymax, xmax), label in zip(pred_scores, pred_boxes, pred_labels):
        if score < 0.5:
            continue
        if labels[label] == 'person':
            # Translate the box coordinates to the masked image coordinates
            ymin -= 240
            ymax -= 240
            xmin -= 240
            xmax -= 240
            if (xmin >= 0) and (ymin >= 0) and (xmax <= 240) and (ymax <= 240):
                person_boxes.append((ymin, xmin, ymax, xmax))
                person_labels.append(labels[label])

    # loop throughout the person detections and place a box around it
    img_boxes = frame.copy()  # Create a copy of the frame for drawing boxes
    for box, label in zip(person_boxes, person_labels):
        ymin, xmin, ymax, xmax = box
        # Translate the box coordinates to the original frame coordinates
        ymin += 240
        ymax += 240
        xmin += 240
        xmax += 240
        img_boxes = cv2.rectangle(img_boxes, (xmin, ymin), (xmax, ymax), (0, 255, 0), 1)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img_boxes, label, (xmin, ymax - 10), font, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('black and white', img_boxes)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
       
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
